# Binomial options pricing model.

This is a notebook about the Binomial Options Pricing model. Our main reference is "Introduction to the Mathematics of Finance" by Steven Roman (2nd Edition, Springer, 2012).

## Formulation

Variables:

* $u$ is the up-tick factor
* $d$ is the down-tick factor
* $r$ is the (constant) risk-free interest rate
* $S_0$ is the initial value of the stock
* $K$ is the strike price of an option
* $T$ is the number of time intervals
* $\Delta t$ is the size of a single time interval

For a real number $x$, let $(x)^+ = \mathrm{ReLU}(x) = \mathrm{max}(0,x)$. The initial value of a call option (resp. put option) with strike price $K$ and expiry date after $T \Delta t$ units of time is given by:

$$ \mathcal I_0(\mathrm{call}) = e^{-rT\Delta t} \sum_{k=0}^T { n \choose k} \left( S_0 u^k d^{T-k} - K\right)^+ \pi^k (1-\pi)^{T-k},$$

$$ \mathcal I_0(\mathrm{put}) = e^{-rT\Delta t} \sum_{k=0}^T { n \choose k} \left( K - S_0 u^k d^{T-k} \right)^+ \pi^k (1-\pi)^{T-k},$$

where $\pi$ is the risk-free (or martingale) up-tick probability, and is given by 

$$\pi = \frac{e^{r\Delta t} - d}{u-d}.$$

The model is free of arbitrage if and only if $d < e^{r \Delta t} < u$. We also have: $1-\pi = \frac{u - e^{r \Delta t}}{u-d}$. Note that the actual, real-world probability of an up-tick or down-tick are not relevant for computing the initial value of a call option. 

### Special case: $T = 1$

When there is only one time step, we have:

$$ \mathcal I_0(\mathrm{call}) = e^{-r\Delta t} \left(  \left(u S_0  - K\right)^+ \pi +  \left(d S_0 - K \right)^+ (1-\pi) \right),$$

$$ \mathcal I_0(\mathrm{put}) = e^{-r\Delta t} \left(  \left(K - u S_0 \right)^+ \pi +  \left(K - d S_0  \right)^+ (1-\pi) \right).$$

For a single time step, it is generally the case that $dS_0 < K < u S_0$, so that the equations reduce to:

$$ \mathcal I_0(\mathrm{call}) = e^{-r\Delta t}  \left(u S_0  - K\right) \pi, \qquad \qquad \mathcal I_0(\mathrm{put}) = e^{-r\Delta t}   \left(K - d S_0  \right) (1-\pi).$$


### Special case: $ud = 1$

The martingale up-tick probability becomes: $\pi = \frac{u e^{r \Delta t} - 1 }{u^2 - 1}$. Also $S_0 u^k d^{T-k} = S_0 u^{2k - T}$.  If, in addtion, the interest rate is zero, then $\pi = \frac{1}{u+1}$ and $1 - \pi = \frac{u}{u+1}$, so the formulas reduce to:

$$ \mathcal I_0(\mathrm{call}) = \frac{1}{(u+1)^{T+1}} \sum_{k=0}^T { n \choose k} \left( S_0 u^{2k-T} - K\right)^+ u^{T-k},$$

$$ \mathcal I_0(\mathrm{put}) = \frac{1}{(u+1)^{T+1}}  \sum_{k=0}^T { n \choose k} \left( K - S_0 u^{2k-T} \right)^+  u^{T-k},$$


<!--

### References

* "Introduction to the Mathematics of Finance: Arbitrage and Options Pricing" by Steven Roman
* https://tv-prod.s3.amazonaws.com/documents%2Fnull-Binomial+Option+Pricing+_f-0943_.pdf
* https://demonstrations.wolfram.com/BinomialOptionPricingModel/
* https://en.wikipedia.org/wiki/Binomial_options_pricing_model
-->

## Code

In [1]:
from math import exp

def choose(k,n):
    a = 1
    for i in range(k):
        a *= (n-i)/(i+1)
    return a

def relu(x):
    return max(0,x)

In [2]:
def prob(u,d,r, dt):
    interest = exp(r*dt)
    assert d < interest and interest < u, "There is no probability eliminating arbitrage"
    return (interest - d)/(u-d)

def initial_call_price(u, d, S0, K, dt, T, r):
    pi = prob(u,d,r, dt)
    answer = 0
    for k in range(T+1):
        Sfinal = S0*(u**k)*(d**(T-k))
        probability = (pi**k)*((1-pi)**(T-k))
        answer += exp(-r*dt*T)*choose(k,T)*relu(Sfinal - K)*probability
    return answer

def initial_put_price(u, d, S0, K, dt, T, r):
    pi = prob(u,d,r, dt)
    answer = 0
    for k in range(T+1):
        Sfinal = S0*(u**k)*(d**(T-k))
        probability = (pi**k)*((1-pi)**(T-k))
        answer += exp(-r*dt*T)*choose(k,T)*relu(K - Sfinal)*probability
    return answer

## Examples and Exercises

<!-- # Check the choose function by printing the first few rows of Pascal's triangle

for n in range(15):
    print()
    for k in range(n+1):
        print(int(choose(k,n)), end = " ")-->

### Example 6.1 from Roman

("Introduction to the Mathematics of Finance" by Steven Roman. 2nd Edition, Springer, 2012.)

In [3]:
pri = prob(1.01, 0.99, 0.01/12, 1)
print(f"Up-tick probability: \t {round(pri, 4)}")
print()

Ks = [102,101, 100, 99, 98, 97]

for K in Ks:
    print(f"Initial call option price with strike price {K} : \t {round(initial_call_price(1.01, 0.99, 100, K, 1, 2, 0.01/12),4)}")

print()
for K in Ks:
    print(f"Initial put option price with strike price {K} : \t {round(initial_put_price(1.01, 0.99, 100, K, 1, 2, 0.01/12),4)}")


Up-tick probability: 	 0.5417

Initial call option price with strike price 102 : 	 0.0029
Initial call option price with strike price 101 : 	 0.2959
Initial call option price with strike price 100 : 	 0.5888
Initial call option price with strike price 99 : 	 1.3725
Initial call option price with strike price 98 : 	 2.1632
Initial call option price with strike price 97 : 	 3.1615

Initial put option price with strike price 102 : 	 1.8331
Initial put option price with strike price 101 : 	 1.1277
Initial put option price with strike price 100 : 	 0.4223
Initial put option price with strike price 99 : 	 0.2076
Initial put option price with strike price 98 : 	 0.0
Initial put option price with strike price 97 : 	 0.0


### Exercise 6.1 in Roman

In [4]:
pri = prob(1.1, 0.9, 0.01/12, 1)
print(f"Up-tick martingale probability: \t {round(pri, 4)}")
print()

Ks = [52, 51, 50, 49, 48, 47]

for K in Ks:
    print(f"Initial call option price with strike price {K}: \t {round(initial_call_price(1.1, 0.9, 50, K, 1, 2, 0.01/12),4)}")

print()
for K in Ks:
    print(f"Initial put option price with strike price {K}: \t \t {round(initial_put_price(1.1, 0.9, 50, K, 1, 2, 0.01/12),4)}")

Up-tick martingale probability: 	 0.5042

Initial call option price with strike price 52: 	 2.157
Initial call option price with strike price 51: 	 2.4107
Initial call option price with strike price 50: 	 2.6645
Initial call option price with strike price 49: 	 3.1678
Initial call option price with strike price 48: 	 3.9207
Initial call option price with strike price 47: 	 4.6736

Initial put option price with strike price 52: 	 	 4.0704
Initial put option price with strike price 51: 	 	 3.3258
Initial put option price with strike price 50: 	 	 2.5812
Initial put option price with strike price 49: 	 	 2.0862
Initial put option price with strike price 48: 	 	 1.8408
Initial put option price with strike price 47: 	 	 1.5954


### Exercise 6.2 in Roman

In [5]:
pri = prob(1.05, 0.9, 0.01/12, 1)
print(f"Up-tick probability: \t {round(pri, 4)}")
print()

Ks = [12, 11, 10, 9, 8]

for K in Ks:
    print(f"Initial call option price with strike price {K}: {round(initial_call_price(1.05, 0.9, 10, K, 1, 2, 0.01/12),4)}")

print()
for K in Ks:
    print(f"Initial put option price with strike price {K}: {round(initial_put_price(1.05, 0.9, 10, K, 1, 2, 0.01/12),4)}")

Up-tick probability: 	 0.6722

Initial call option price with strike price 12: 0.0
Initial call option price with strike price 11: 0.0113
Initial call option price with strike price 10: 0.4624
Initial call option price with strike price 9: 1.1115
Initial call option price with strike price 8: 2.0133

Initial put option price with strike price 12: 1.98
Initial put option price with strike price 11: 0.993
Initial put option price with strike price 10: 0.4458
Initial put option price with strike price 9: 0.0965
Initial put option price with strike price 8: 0.0


In [6]:
## Variation wth a higher u and lower d

pri = prob(1.1, 0.95, 0.01/12, 1)
print(f"Up-tick martingale probability: \t {round(pri, 4)}")
print()

Ks = [12, 11, 10, 9, 8]

for K in Ks:
    print(f"Initial call option price with strike price {K}: {round(initial_call_price(1.1, 0.95, 10, K, 1, 2, 0.01/12),4)}")

print()
for K in Ks:
    print(f"Initial put option price with strike price {K}: {round(initial_put_price(1.1, 0.95, 10, K, 1, 2, 0.01/12),4)}")

Up-tick martingale probability: 	 0.3389

Initial call option price with strike price 12: 0.0115
Initial call option price with strike price 11: 0.1261
Initial call option price with strike price 10: 0.4421
Initial call option price with strike price 9: 1.015
Initial call option price with strike price 8: 2.0133

Initial put option price with strike price 12: 1.9915
Initial put option price with strike price 11: 1.1078
Initial put option price with strike price 10: 0.4254
Initial put option price with strike price 9: 0.0
Initial put option price with strike price 8: 0.0


### Exercise 6.3 in Roman

In [7]:
u = 1.01
d = 0.99

# The possible values are:
print(f"The possible values are: \n\t {100*u*u} \n\t {100*u*d} \n\t {100*d*u} \n\t {100*d*d} \n")

print(f"Three of these four values are below 100.")
print(f"Assuming equal real-world up-tick and down-tick probabilities, the probability of a loss is 3/4.")

The possible values are: 
	 102.01 
	 99.99 
	 99.99 
	 98.01 

Three of these four values are below 100.
Assuming equal real-world up-tick and down-tick probabilities, the probability of a loss is 3/4.


### Problem 2.3.2 in Stefanica, Radoicic, and Wang

("150 Most Frequently Asked Questions on Quant Interviews", by Stefanica, Radocic, and Wang. FE Press, 2019.)

In [8]:
print(f"Initial put option price with strike price K= 50: {round(initial_put_price(52/50, 47/50, 50, 50, 1, 1, 0),4)}")

print(f"Up-tick martingale probability: {round( prob(52/50, 47/50, 0, 1), 4)}")
print(f"Down-tick martingale probability: {round( 1- prob(52/50, 47/50, 0, 1), 4)}")

Initial put option price with strike price K= 50: 1.2
Up-tick martingale probability: 0.6
Down-tick martingale probability: 0.4


# Hedged portfolio

We focus on the $T=1$ case and develop a hedged portfolio. Namely, we buy $H$ units of the asset and sell one call option with expiry date $\Delta t = 1$ and strike price $K$. The constant $H$ is between 0 and 1, and is known as the hedge ratio, and is chosen so that we earn the same profit whether or not the price of the asset goes up or down. 

To compute the desired hedge ratio, note that our initial portfolio has value $HS_0 - C$, where $C$ is the initial cost of the call option. If there is an up-tick, it has value $uHS_0 - C_u$, where $C_u = \textrm{max}(0, uS_0 - K)$ is the new value of the call option. If there is an uptick, our portfolio has value $dHS_0 - C_d$, where $C_d = \textrm{max}(0, dS_0 - K)$ is the new value of the call option. Setting these two quantities to be equal and solving for $H$, we obtain:

$$H = \frac{C_u - C_d}{(u-d)S_0}.$$

In order to avoid vacuous cases, we generally have $dS_0 < K < uS_0$, in which case the hedge ratio becomes:

$$H = \frac{u - K/S_0}{u-d}.$$

To compute the initial cost of the call option, we set the initial portfolio value to that after a down-tick, i.e. $HS_0 - C = dHS_0$. (Equivalently, we could have set it equal to the value after an up-tick.) Sovling for $C$, we obtain:

$$C = H(S_0 - dS_0) = \frac{\left( uS_0 - K \right) \left( 1-d\right)}{u-d}$$

In [9]:
def hedge_ratio(u, d, S0, K):
    return (u - K/S0)/(u-d)

def call_option_value(u, d, S0, K):
    return (u*S0 - K)*(1-d)/(u-d)